# Logistic Regression with a Neural Network mindset

* 通过逻辑回归实现分类器进行识别
* 选用kernel learning_dl
~~~
conda activate learning_dl 

# conda remove --name learning_dl --all
# kill -9 +id
~~~

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
from lr_utils import load_dataset

%matplotlib inline

print("finish import module!!!")

finish import module!!!


In [3]:
# Loading the data (cat/non-cat)
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

# 显示第十张图片
# # Example of a picture
# index =10
# plt.imshow(train_set_x_orig[index])
# print ("y = " + str(train_set_y[0, index]) + ", it's a '" + classes[np.squeeze(train_set_y[:, index])].decode("utf-8") +  "' picture.")

OSError: Unable to open file (file signature not found)